# Segment, localise, downscale and track

Downscaling of images to fit within 1200x1200 is a janky fix to improve tracking significantly 

In [15]:
import napari
from cellpose import models
from octopuslite import utils, tile
import numpy as np
import os

import sys
sys.path.append('macrohet/')
from notify import send_sms

def view(img):
    return napari.Viewer().add_image(img)

from tqdm.auto import tqdm

import btrack
import dask.array as da

import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
model = models.Cellpose(
                        #gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )
from skimage.transform import rescale, resize, downscale_local_mean
import sys
sys.path.append('../')
from notify import send_sms

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    32.1 GB


In [24]:
from macrohet import dataio, tile
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
from skimage import transform
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
# defining cellpose model to use
model = models.Cellpose(
                        gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )
# defining tracking config to use
config = '/home/dayn/analysis/BayesianTracker/models/cell_config.json'

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    8.3 GB


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [6]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = dataio.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [8]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True)
dataio.read_harmony_metadata(metadata_path, assay_layout=True)

Reading metadata XML file...
Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Pick one position to test first

In [13]:
row, column = (3,5)

# Segment, localise, downscale and track

In [25]:
### load images into memory?
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)
### define gfp input images for segmentation
input_images = images[:,0,...]
### define empty mask stack to append each mask frame to
masks_stack = list()
### iterate over frames and segment
for n, frame in tqdm(enumerate(input_images), total = len(input_images), 
                     desc = f'Segmenting position {row, column}', leave = False):
    ### decide whether to compute on the fly or not
    frame = frame.compute().compute().astype(np.uint16)
    ### segment
    masks, flows, styles, diams = model.eval(frame, 
                                             channels = [0,0], 
                                             diameter = 325, 
                                             min_size = 2500, 
                                             )
    break
#     masks_stack.append(masks)

mask_resized = transform.rescale(masks,
#                         masks_stack[t], 
                       1200/6048, anti_aliasing=False)
### rescale so that pixel values are 16 bit?
mask_resized *= 2**16/mask_resized.max()  

SyntaxError: invalid syntax (3861544465.py, line 28)

In [57]:
1/( 1200/6048)

5.04

In [58]:
import numpy as np
from skimage.measure import block_reduce

# Load the original image as a uint8 array
# Assuming you have loaded your image into a NumPy array called "image"
# with shape (height, width) and data type uint8

# Define the block size for downsampling
block_size = (6048/1200, 6048/1200)

# Use block_reduce to downsample the image by a factor of 5
# and preserve the pixel values by taking the mode of the blocks
resized_image = block_reduce(masks, block_size, func=np.max)

# Convert the resized image back to uint8
resized_image = resized_image.astype(np.uint8)

# The resized_image array now contains the resized image with preserved shape and pixel values as uint8


TypeError: `pad_width` must be of integral type.

In [ ]:
import numpy as np
from skimage.transform import rescale

# Load the original image as a uint8 array
# Assuming you have loaded your image into a NumPy array called "image"
# with shape (height, width) and data type uint8

# Define the scaling factor
scale_factor = 5.04

# Use skimage.transform.rescale to downscale the image by the scale factor
resized_image = rescale(masks, scale_factor, order=0, preserve_range=True)

# Convert the resized image back to uint8
resized_image = resized_image.astype(np.uint16)

# The resized_image array now contains the resized image with preserved shape and pixel values as uint8


In [39]:

mask_resized = transform.rescale(masks,
#                         masks_stack[t], 
                       1200/6048, anti_aliasing=True)
### rescale so that pixel values are 16 bit?
resized_uint16 = (mask_resized * 65535).astype(np.uint16)

# mask_resized *= 2**16/mask_resized.max()  

In [22]:
import napari

In [54]:
v = napari.Viewer()

v.add_labels(masks)
v.add_image(mask_resized, name = 'original')

v.add_labels(resized_image, name = 'chat')

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Labels layer 'chat' at 0x7f783ac15c10>

In [ ]:
### load images into memory?
images = tile.compile_mosaic(
                             image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)
### define gfp input images for segmentation
input_images = images[:,0,...]
### define empty mask stack to append each mask frame to
masks_stack = list()
### iterate over frames and segment
for n, frame in tqdm(enumerate(input_images), total = len(input_images), 
                     desc = f'Segmenting position {row, column}', leave = False):
    ### decide whether to compute on the fly or not
    frame = frame.compute().compute().astype(np.uint16)
    ### segment
    masks, flows, styles, diams = model.eval(
                                             frame, 
                                             channels = [0,0], 
                                             diameter = 325, 
                                             min_size = 2500, 
                                             )
    masks_stack.append(masks)
### make a stack out of this using dask
masks_stack = da.stack(masks_stack, axis = 0)
### reorder the channel axis so that localisation can measure intensity
gfp = images[:,0,...]
rfp = images[:,1,...]
#     ### stack together using dask 
#     images = da.stack([gfp,rfp], axis = -1)
#     ### locaslise objects and measure properties
#     objects = btrack.utils.segmentation_to_objects(
#                                                     masks_stack, 
#                                                     images,
#                                                     properties = tuple(feat),
#                                                     use_weighted_centroid = False, 
#                                                     )
#     ### filter small objects out 
#     objects = [o for o in objects if o.properties['area'] > 2500]
#     ### save objects out as insurance
#     with btrack.dataio.HDF5FileHandler(
#                                        f'/mnt/DATA/macrohet/segmentation/({row},{column})_objects_d325.h5',
#                                        'w', 
#                                        obj_type='obj_type_1',
#                                        ) as hdf:
#                                         hdf.write_segmentation(masks_stack)
#                                         hdf.write_objects(objects)
### define new array for resized masks
masks_stack_resized = list()
### now conduct rescaling and save out as seperate objects prior to tracking
for t in tqdm(range(len(masks_stack)), 
              desc = f'Resizing masks in position {row, column}', leave = False):
    ### resize mask to be 1200 pixels squared
    mask_resized = rescale(masks_stack[t], 1200/6048, anti_aliasing=False)
    ### rescale so that pixel values are 16 bit?
    mask_resized *= 2**16/mask_resized.max()   
    masks_stack_resized.append(mask_resized.astype(np.uint16))
masks_stack_resized = da.stack(masks_stack_resized, axis = 0)
### now resize the gfp and rfp
gfp_images_resized = list()
for t in tqdm(range(len(gfp)), 
              desc = f'Resizing GFP images in position {row, column}', leave = False):
    gfp_image_resized = rescale(gfp[t], 1200/6048, anti_aliasing=False)
    gfp_images_resized.append(gfp_image_resized)
gfp_images_resized = da.stack(gfp_images_resized, axis = 0)
rfp_images_resized = list()
for t in tqdm(range(len(rfp)),  
              desc = f'Resizing RFP images in position {row, column}', leave = False):
    rfp_image_resized = rescale(rfp[t], 1200/6048, anti_aliasing=False)
    rfp_images_resized.append(rfp_image_resized)
rfp_images_resized = da.stack(rfp_images_resized, axis = 0)
### stack together resized images for localisation
resized_images = da.stack([gfp_images_resized,rfp_images_resized], axis = -1)
### define features to measure 
feat = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]
### localise resized images
objects = btrack.utils.segmentation_to_objects(
                                                masks_stack_resized, 
                                                resized_images,
                                                properties = tuple(feat),
                                                use_weighted_centroid = False, 
                                            )
### filter out small objects
objects = [o for o in objects if o.properties['area'] > 500]
### save out resized objects
with btrack.dataio.HDF5FileHandler(
                                    f'/mnt/DATA/macrohet/segmentation/({row},{column})_objects_rescaled.h5',#
                                    'w', 
                                    obj_type='obj_type_1',
                                    ) as hdf:
                                    hdf.write_segmentation(masks_stack_resized)
                                    hdf.write_objects(objects)
### redefine features so that both channels are included in track measurements
feat = list(objects[0].properties.keys())
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure(config)
    tracker.max_search_radius = 20
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    tracker.features = feat

    # append the objects to be tracked
    tracker.append(objects)

    # set the tracking volume
    tracker.volume=((0, masks_stack_resized.shape[-2]), (0, masks_stack_resized.shape[-1]))

    # track them (in interactive mode)
    tracker.track(step_size=25)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari()

    # store the tracks
    tracks = tracker.tracks

    # store the configuration
    cfg = tracker.configuration

    # export the track data 
    tracker.export(f"/mnt/DATA/macrohet/segmentation/({row},{column})_tracks_rescaled.h5", obj_type="obj_type_1")


In [16]:
### define positions to do
positions = [(3,8), (4,5), (5,4), (5,8)]
### iterate over positions
# for row, column in tqdm(positions, desc = f'Progress through different positions'):
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    if os.path.exists(f"/mnt/DATA/macrohet/segmentation/({row},{column})_tracks_rescaled.h5"):
        print(f'Position {row, column} already completed')
        continue
    ### load images into memory?
    images = tile.compile_mosaic(
                                 image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)
    ### define gfp input images for segmentation
    input_images = images[:,0,...]
    ### define empty mask stack to append each mask frame to
    masks_stack = list()
    ### iterate over frames and segment
    for n, frame in tqdm(enumerate(input_images), total = len(input_images), 
                         desc = f'Segmenting position {row, column}', leave = False):
        ### decide whether to compute on the fly or not
        frame = frame.compute().compute().astype(np.uint16)
        ### segment
        masks, flows, styles, diams = model.eval(
                                                 frame, 
                                                 channels = [0,0], 
                                                 diameter = 325, 
                                                 min_size = 2500, 
                                                 )
        masks_stack.append(masks)
    ### make a stack out of this using dask
    masks_stack = da.stack(masks_stack, axis = 0)
    ### reorder the channel axis so that localisation can measure intensity
    gfp = images[:,0,...]
    rfp = images[:,1,...]
#     ### stack together using dask 
#     images = da.stack([gfp,rfp], axis = -1)
#     ### locaslise objects and measure properties
#     objects = btrack.utils.segmentation_to_objects(
#                                                     masks_stack, 
#                                                     images,
#                                                     properties = tuple(feat),
#                                                     use_weighted_centroid = False, 
#                                                     )
#     ### filter small objects out 
#     objects = [o for o in objects if o.properties['area'] > 2500]
#     ### save objects out as insurance
#     with btrack.dataio.HDF5FileHandler(
#                                        f'/mnt/DATA/macrohet/segmentation/({row},{column})_objects_d325.h5',
#                                        'w', 
#                                        obj_type='obj_type_1',
#                                        ) as hdf:
#                                         hdf.write_segmentation(masks_stack)
#                                         hdf.write_objects(objects)
    ### define new array for resized masks
    masks_stack_resized = list()
    ### now conduct rescaling and save out as seperate objects prior to tracking
    for t in tqdm(range(len(masks_stack)), 
                  desc = f'Resizing masks in position {row, column}', leave = False):
        ### resize mask to be 1200 pixels squared
        mask_resized = rescale(masks_stack[t], 1200/6048, anti_aliasing=False)
        ### rescale so that pixel values are 16 bit?
        mask_resized *= 2**16/mask_resized.max()   
        masks_stack_resized.append(mask_resized.astype(np.uint16))
    masks_stack_resized = da.stack(masks_stack_resized, axis = 0)
    ### now resize the gfp and rfp
    gfp_images_resized = list()
    for t in tqdm(range(len(gfp)), 
                  desc = f'Resizing GFP images in position {row, column}', leave = False):
        gfp_image_resized = rescale(gfp[t], 1200/6048, anti_aliasing=False)
        gfp_images_resized.append(gfp_image_resized)
    gfp_images_resized = da.stack(gfp_images_resized, axis = 0)
    rfp_images_resized = list()
    for t in tqdm(range(len(rfp)),  
                  desc = f'Resizing RFP images in position {row, column}', leave = False):
        rfp_image_resized = rescale(rfp[t], 1200/6048, anti_aliasing=False)
        rfp_images_resized.append(rfp_image_resized)
    rfp_images_resized = da.stack(rfp_images_resized, axis = 0)
    ### stack together resized images for localisation
    resized_images = da.stack([gfp_images_resized,rfp_images_resized], axis = -1)
    ### define features to measure 
    feat = [
      "area",
      "major_axis_length",
      "minor_axis_length",
      "orientation",
      "mean_intensity",
        ]
    ### localise resized images
    objects = btrack.utils.segmentation_to_objects(
                                                    masks_stack_resized, 
                                                    resized_images,
                                                    properties = tuple(feat),
                                                    use_weighted_centroid = False, 
                                                )
    ### filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]
    ### save out resized objects
    with btrack.dataio.HDF5FileHandler(
                                        f'/mnt/DATA/macrohet/segmentation/({row},{column})_objects_rescaled.h5',#
                                        'w', 
                                        obj_type='obj_type_1',
                                        ) as hdf:
                                        hdf.write_segmentation(masks_stack_resized)
                                        hdf.write_objects(objects)
    ### redefine features so that both channels are included in track measurements
    feat = list(objects[0].properties.keys())
    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure(config)
        tracker.max_search_radius = 20
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        tracker.features = feat

        # append the objects to be tracked
        tracker.append(objects)

        # set the tracking volume
        tracker.volume=((0, masks_stack_resized.shape[-2]), (0, masks_stack_resized.shape[-1]))

        # track them (in interactive mode)
        tracker.track(step_size=25)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        # get the tracks in a format for napari visualization
        data, properties, graph = tracker.to_napari()

        # store the tracks
        tracks = tracker.tracks

        # store the configuration
        cfg = tracker.configuration

        # export the track data 
        tracker.export(f"/mnt/DATA/macrohet/segmentation/({row},{column})_tracks_rescaled.h5", obj_type="obj_type_1")
    
    ### notify job is finished
    send_sms(f'Position {row},{column} tracked')

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

Segmenting position (3, 4):   0%|          | 0/75 [00:00<?, ?it/s]

Resizing masks in position (3, 4):   0%|          | 0/75 [00:00<?, ?it/s]

Resizing GFP images in position (3, 4):   0%|          | 0/75 [00:00<?, ?it/s]

Resizing RFP images in position (3, 4):   0%|          | 0/75 [00:00<?, ?it/s]

...

Resizing GFP images in position (6, 9):   0%|          | 0/75 [00:00<?, ?it/s]

Resizing RFP images in position (6, 9):   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/01/29 07:47:37 AM] Localizing objects from segmentation...
[INFO][2023/01/29 07:47:37 AM] Found intensity_image data
[INFO][2023/01/29 07:51:37 AM] Objects are of type: <class 'dict'>
[INFO][2023/01/29 07:51:49 AM] ...Found 1087113 objects in 75 frames.
[INFO][2023/01/29 07:51:50 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/(6,9)_objects_rescaled.h5...
[INFO][2023/01/29 07:51:54 AM] Writing objects/obj_type_1
[INFO][2023/01/29 07:51:54 AM] Writing labels/obj_type_1
[INFO][2023/01/29 07:51:54 AM] Loading objects/obj_type_1 (27816, 5) (27816 filtered: None)
[INFO][2023/01/29 07:51:54 AM] Writing properties/obj_type_1/area (27816,)
[INFO][2023/01/29 07:51:54 AM] Writing properties/obj_type_1/major_axis_length (27816,)
[INFO][2023/01/29 07:51:54 AM] Writing properties/obj_type_1/minor_axis_length (27816,)
[INFO][2023/01/29 07:51:54 AM] Writing properties/obj_type_1/orientation (27816,)
[INFO][2023/01/29 07:51:54 AM] Writing properties/obj_type_1/mean_intensity-0 (27816

GLPK Integer Optimizer 5.0
10040 rows, 8522 columns, 12121 non-zeros
8522 integer variables, all of which are binary
Preprocessing...
5020 rows, 8522 columns, 12121 non-zeros
8522 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5020
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5020 rows, 8522 columns, 12121 non-zeros
*     0: obj =   2.620848664e+04 inf =   0.000e+00 (2152)
Perturbing LP to avoid stalling [941]...
Removing LP perturbation [2118]...
*  2118: obj =   1.391599054e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2118: mip =     not found yet >=              -inf        (1; 0)
+  2119: >>>>>   1.391732918e+04 >=   1.391732918e+04   0.0% (2; 0)
+  2119: mip =   1.391732918e+04 >=     tree is empty   0.0% (0; 3)
INTEGER OPTIMAL SOLU

[INFO][2023/01/29 07:52:07 AM] Opening HDF file: /mnt/DATA/macrohet/segmentation/(6,9)_tracks_rescaled.h5...
[INFO][2023/01/29 07:52:07 AM] Writing objects/obj_type_1
[INFO][2023/01/29 07:52:07 AM] Writing labels/obj_type_1
[INFO][2023/01/29 07:52:07 AM] Loading objects/obj_type_1 (27816, 5) (27816 filtered: None)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/area (27816,)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/major_axis_length (27816,)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/minor_axis_length (27816,)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/orientation (27816,)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/mean_intensity-0 (27816,)
[INFO][2023/01/29 07:52:07 AM] Writing properties/obj_type_1/mean_intensity-1 (27816,)
[INFO][2023/01/29 07:52:08 AM] Writing tracks/obj_type_1
[INFO][2023/01/29 07:52:08 AM] Writing dummies/obj_type_1
[INFO][2023/01/29 07:52:08 AM] Writing LBEP/obj_type_1
[INFO][

In [57]:
v = napari.Viewer()
v.add_image(images, 
            channel_axis = -1, 
            colormap=['green', 'magenta'], 
            contrast_limits=[[100,6000],[0,3000]])
# for key in masks_dict.keys():
#     v.add_labels(masks_dict[key][0], name = key)
v.add_labels(
    masks, 
    name = 'masks', 
#     color = {1:'yellow'}, 
    opacity = 0.2
)
v.add_tracks(
                    data, 
                    properties=properties, 
                    graph=graph, 
                    name="Tracks 5,4", 
                    blending="translucent",
                    visible=False,
                )


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Tracks layer 'Tracks 5,4' at 0x7f41173612b0>